In [9]:
!pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.9.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: azure-common, azure-storage-file-share, isodate, msrest, pydash, strictyaml, pyjwt, opencensus-ext-azure, marshmallow, azure-core, pyyaml, tqdm, azure-storage-blob, azure-storage-file-datalake, typing-extensions, azure-mgmt-core, colorama, jsonschema
Required-by: 


In [10]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [11]:
ml_client=MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [12]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime


online_endpoint_name="endpoint-"+datetime.datetime.now().strftime("%m%d%H%M%f")

# Create an online endpoint
endpoint=ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online Endpoint for MLFLOW diabetes model",
    auth_mode="key"
    )

<h3> <a>https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints?view=azureml-api-2&wt.mc_id=aiml-17954-sejuare&tabs=python </a></h3>

In [13]:
# Create the endpoint
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-08100516018674.westus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-08100516018674.westus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-08100516018674', 'description': 'Online Endpoint for MLFLOW diabetes model', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/resourcegroups/rg-dp100-la4e97039a08e47518e/providers/microsoft.machinelearningservices/workspaces/mlw-dp100-la4e97039a08e47518e/onlineendpoints/endpoint-08100516018674', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/providers/Microsoft.MachineLearningServices/locations/westus/mfeOperationsStatus/oe:483d469a-b0ea-488d-8ffe-c1fe9008d232:5cf5c775-2ca7-4485-8ee7-ff56c1a75c5d?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/18a1f27f-

In [14]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

# create a blue deployment
model=Model(
    path="./model",
    type=AssetTypes.MLFLOW_MODEL,
    description="My Sample mlflow model"
    )

blue_deployment=ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model, 
    instance_type="Standard_F4s_v2",
    instance_count=1
    )


In [15]:
ml_client.online_deployments.begin_create_or_update(deployment=blue_deployment).result()

Check: endpoint endpoint-08100516018674 exists
Uploading model (0.0 MBs): 100%|██████████| 2424/2424 [00:00<00:00, 15179.17it/s]




....................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpoint-08100516018674', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/providers/Microsoft.MachineLearningServices/locations/westus/mfeOperationsStatus/od:483d469a-b0ea-488d-8ffe-c1fe9008d232:1fbe6121-6911-4948-9c2b-4d2827cf75e5?api-version=2023-04-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/resourceGroups/rg-dp100-la4e97039a08e47518e/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-la4e97039a08e47518e/onlineEndpoints/endpoint-08100516018674/deployments/blue', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cia4e97039a08e47518e/code/Users/ritishadhikari/azure-ml-labs/Labs/11', 'creation_context': None, 'serialize': <msre

In [16]:
endpoint.traffic={"blue":100}

ml_client.begin_create_or_update(entity=endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-08100516018674.westus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-08100516018674.westus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-08100516018674', 'description': 'Online Endpoint for MLFLOW diabetes model', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/resourcegroups/rg-dp100-la4e97039a08e47518e/providers/microsoft.machinelearningservices/workspaces/mlw-dp100-la4e97039a08e47518e/onlineendpoints/endpoint-08100516018674', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/providers/Microsoft.MachineLearningServices/locations/westus/mfeOperationsStatus/oe:483d469a-b0ea-488d-8ffe-c1fe9008d232:f3261168-dd73-46d5-bab1-9e0bc4e275af?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/18a1f27f-

<h3> Test the Deployment</h3>

In [21]:
response=ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json"
)

if response[1]=="1":
    print("Diabetic")
else:
    print("Not Diabetic")

[1]
Diabetic


<h3> List all endpoints </h3>



In [22]:
endpoints=ml_client.online_endpoints.list()

for endp in endpoints:
    print(endp.name)

endpoint-08100454495046
endpoint-08100516018674


In [24]:
endpoint=ml_client.online_endpoints.get(name=online_endpoint_name)

# get existing traffic details
print(endpoint.traffic)

# get scoring uri
print(endpoint.scoring_uri)

{'blue': 100}
https://endpoint-08100516018674.westus.inference.ml.azure.com/score


<h3> Delete the endpoint </h3>



In [25]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

........